# Searching for Specific Moths

In this notebook, we will search for specific moths in a dataset. Specifically, those which are: 
- large
- sloth moths
- highest test accuracy

In [1]:
import os
import pandas as pd
import boto3
import json
from boto3.s3.transfer import TransferConfig
from PIL import Image
import numpy as np
from tqdm import tqdm

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# set the working directory
os.chdir(os.path.expanduser('~/amber-inferences'))

In [3]:
region='sgp'
country='singapore'
download_dir=f'./data/qc_plots/{country}'
os.makedirs(download_dir, exist_ok=True)

inference_dir = os.path.abspath(f'/gws/nopw/j04/ceh_generic/kgoldmann/{country}_inferences_tracking/')

#listdir recursively
def listdir_recursive(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            yield os.path.join(root, file)

# Get all csv files in the inference directory
inference_csvs = list(listdir_recursive(inference_dir))
inference_csvs = [c for c in inference_csvs if c.endswith('.csv')]
inference_csvs = [c for c in inference_csvs if not 'compute' in c]

In [4]:
deps = [os.path.basename(os.path.dirname(x)) for x in inference_csvs]

# print the value counts
print(pd.Series(deps).value_counts())

dep000050    134
dep000049    104
dep000045    100
dep000052     99
dep000054     79
dep000053     44
dep000051     39
dep000048     25
dep000046     18
dep000047     16
Name: count, dtype: int64


In [5]:
len(inference_csvs)

658

## Plotting and Data Wrangling Functions

In [6]:
def download_images(s3_client, config, key, download_dir, bucket_name):
    download_path = os.path.join(download_dir, os.path.basename(key))
    s3_client.download_file(bucket_name, key, download_path, Config=config)

In [7]:
def initialise_session(credentials_file="credentials.json"):
    """
    Load AWS and API credentials from a configuration file and initialise an AWS session.

    Args:
        credentials_file (str): Path to the credentials JSON file.

    Returns:
        boto3.Client: Initialised S3 client.
    """
    with open(credentials_file, encoding="utf-8") as config_file:
        aws_credentials = json.load(config_file)
    session = boto3.Session(
        aws_access_key_id=aws_credentials["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=aws_credentials["AWS_SECRET_ACCESS_KEY"],
        region_name=aws_credentials["AWS_REGION"],
    )
    client = session.client("s3", endpoint_url=aws_credentials["AWS_URL_ENDPOINT"])
    return client

client = initialise_session('./credentials.json')

In [8]:
# Transfer configuration for optimised S3 download
transfer_config = TransferConfig(
    max_concurrency=20,  # Increase the number of concurrent transfers
    multipart_threshold=8 * 1024 * 1024,  # 8MB
    max_io_queue=1000,
    io_chunksize=262144,  # 256KB
)

In [9]:
def subset_by_species(inference_csvs, species_names, top_n=1, confidence_threshold=0):
    df_moths = pd.DataFrame()
    for c in tqdm(inference_csvs, desc='reading in the csvs'):
        try:
            input_df = pd.read_csv(c, low_memory=False)

        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue

        subset_df = pd.DataFrame()
        for i in range(top_n):
            col_name = f'top_{i+1}_species'
            if col_name not in input_df.columns:
                print(f" - Column {col_name} not found in {c}. Skipping this file.")
                continue
            temp = input_df.loc[(input_df[col_name].isin(species_names)) &
                                (input_df[col_name.replace('species', 'confidence')] > confidence_threshold), ]
            subset_df = pd.concat([subset_df, temp], ignore_index=True)
        prefix = os.path.basename(os.path.dirname(c)) + "/snapshot_images/"
        subset_df['key'] =  subset_df['image_path'].apply(lambda x: f"{prefix}{os.path.basename(x)}")

        df_moths = pd.concat([df_moths, subset_df], ignore_index=True)
        del subset_df
        del input_df
    return df_moths

In [ ]:
def subset_by_size(inference_csvs, keep_n=20, moth_only=True, size_cutoff=None):
    df_moths = pd.DataFrame()
    for c in tqdm(inference_csvs, desc='reading in the csvs'):
        try:
            input_df = pd.read_csv(c, low_memory=False)

        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue

        input_df = input_df[input_df['crop_status'].str.contains('crop_')]
        if input_df.empty:
            continue

        input_df = input_df.sort_values('crop_area', ascending=False)

        if moth_only:
            input_df = input_df[(input_df['order_name'].str.contains('Lepidoptera')) | (input_df['class_name'] == 'moth')]

        if size_cutoff is not None:
            input_df = input_df[input_df['crop_area'] > size_cutoff]

        input_df = input_df.head(keep_n)

        prefix = os.path.basename(os.path.dirname(c)) + "/snapshot_images/"
        input_df['key'] =  input_df['image_path'].apply(lambda x: f"{prefix}{os.path.basename(x)}")

        df_moths = pd.concat([df_moths, input_df], ignore_index=True)

        del input_df
    return df_moths

In [35]:
def output_crops(df, output_dir, region, buffer=5):
    for i, row in tqdm(df.iterrows(), desc='downloading images', total=df.shape[0]):
        try:
            download_images(client, transfer_config, row['key'], output_dir, region)
        except Exception as e:
            print(f" - Error downloading {row['key']}: {e}")

        dep = row['deployment_id']

        # crop the image and save it
        image_path = os.path.join(output_dir, str(os.path.basename(row['key'])))
        cropped_image_path = os.path.join(output_dir, f"{dep}_{row['crop_status']}_{os.path.basename(row['key'])}")
        try:
            with Image.open(image_path) as img:
                # Crop the image
                x_min = float(row['x_min']) -buffer
                y_min = float(row['y_min']) -buffer
                x_max = float(row['x_max']) +buffer
                y_max = float(row['y_max']) +buffer

                img_cropped = img.crop((x_min, y_min, x_max, y_max))
                # Save the cropped image
                img_cropped.save(cropped_image_path)
            os.remove(image_path)  # Remove the original image after cropping

        except Exception as e:
            print(f" - Error cropping {image_path}: {e}")

# From Lists Provided 

In [ ]:
# read in the txt file
def read_species_list(file_path):
    with open(file_path, 'r') as file:
        species_list = [line.strip() for line in file if line.strip()]
    return species_list

moths = read_species_list(f'./sandbox/lists/{region}_interesting_moths.txt') # or './sandbox/lists/sloth_moths.txt

moths = [x.replace('nr. ', '') for x in moths]

In [ ]:
all_moths = pd.read_csv(f'../gbif_download_standalone/species_checklists/{country}-moths-keys-nodup.csv')

In [ ]:
# get moths in all_moths['species_name']
moths_in_all = all_moths[all_moths['species_name_provided'].isin(moths)]
moths_missing = [x for x in moths if x not in moths_in_all['species_name_provided'].values]

In [ ]:
moths_missing

In [ ]:
notable_moths = all_moths.loc[(all_moths['gbif_species_name'].isin(moths)) |
                              (all_moths['search_species_name'].isin(moths)) |
                              (all_moths['species_name_provided'].isin(moths)), ]

notable_moths = notable_moths['gbif_species_name'].values.tolist()
notable_moths

In [ ]:
inferenced_moths = subset_by_species(inference_csvs, notable_moths, 1, 0)

In [ ]:
inferenced_moths.head()

In [ ]:
inferenced_moths['top_1_species'].value_counts()

In [ ]:
inferenced_moths['deployment_name'].value_counts().plot(kind='bar',
                            title='Deployment Name Counts',
                            xlabel='Deployment Name',
                            ylabel='Count')


In [ ]:
# plot a historgram
inferenced_moths['top_1_confidence'].plot(kind='hist',
                            title='Confidence Distribution for Interesting Moths',
                            xlabel='Confidence',
                            bins=50)


In [ ]:
plot_df = inferenced_moths[(~inferenced_moths[['deployment_name', 'track_id']].duplicated()) | (inferenced_moths['track_id'].isnull())]

plot_df['top_1_confidence'].plot(kind='hist',
                            title='Confidence Distribution (accounting for tracking)',
                            xlabel='Confidence',
                            color='orange',
                            bins=50)

plt.show()

In [ ]:
inferenced_backup = inferenced_moths.copy()

In [ ]:
inferenced_moths = inferenced_backup

In [ ]:
# Option to remove certain dates

inferenced_moths['date'] = pd.to_datetime(inferenced_moths['image_datetime'])

# subset to date only
inferenced_moths['date'] = inferenced_moths['date'].dt.date

inferenced_moths = inferenced_moths.loc[(inferenced_moths['deployment_name'] != 'W47') &
    (~inferenced_moths['date'].isin([
        datetime.date(2024, 6, 26),
        datetime.date(2024, 6, 27)
    ])
    )
]
inferenced_moths = inferenced_moths.loc[(inferenced_moths['deployment_name'] != 'ECOLINK') &
    (~inferenced_moths['date'].isin([
        datetime.date(2024, 7, 25)
    ])
    )
]

# get the number of occurrences per date
occurrences_per_date_by_deployment = inferenced_moths[['date', 'deployment_name']].value_counts().sort_index()
occurrences_per_date_by_deployment

occurrences_per_date = inferenced_moths['date'].value_counts().sort_index()
occurrences_per_date

In [ ]:
# plot date vs count
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(occurrences_per_date.index, occurrences_per_date.values, marker='o')
plt.title('Occurrences Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# colour by deployment_name
for dep in inferenced_moths['deployment_name'].astype('category').cat.categories:
    dep_mask = inferenced_moths['deployment_name'] == dep
    plt.scatter(inferenced_moths.loc[dep_mask, 'date'], inferenced_moths.loc[dep_mask, 'top_1_confidence'], label=dep)
plt.legend()
plt.title('Occurrences Over Time')
plt.xlabel('Date')
plt.ylabel('Prediction Confidence')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
for deployment_name, group in occurrences_per_date_by_deployment.groupby(level=1):
    plt.plot(group.index.get_level_values(0), group.values, marker='o', label=deployment_name)
plt.title('Occurrences Over Time by Deployment')
plt.xlabel('Date')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45)
plt.grid()
plt.legend(title='Deployment Name')
plt.show()

In [ ]:
download_dir = '/gws/nopw/j04/ceh_generic/kgoldmann/interesting_moths/'

os.makedirs(download_dir, exist_ok=True)

In [ ]:
import datetime

# for each top_1_species get the most confident images for that species
confident_moths = inferenced_moths.sort_values('top_1_confidence', ascending=False)

confident_moths = confident_moths.loc[confident_moths['top_1_confidence'] > 0.8, ]
# confident_moths = confident_moths.groupby('top_1_species').head(1000)
confident_moths.head()

In [ ]:
confident_moths['top_1_species'].value_counts()

In [ ]:
output_crops(confident_moths, download_dir, region, buffer=5)

# High Test Accuracy Moths

In [ ]:
# load in the json files
def load_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

region_list = {
    'costarica':'03',
    'singapore':'02',
    'thailand':'02',
}


with open(f'/home/users/katriona/amber-inferences/sandbox/turing-{country}_v{region_list[country]}_taxon-accuracy.json') as f:
    accuracy = json.load(f)

info = accuracy['About']
(k := next(iter(accuracy)), accuracy.pop(k))

In [ ]:
len(accuracy['species'])

In [ ]:
tax_df_list = {}

for tax in accuracy.keys():
    print(tax)

    tax_acc = accuracy[tax]

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(tax_acc, orient='index', columns=['Top1 Accuracy', 'Total Test Points'])

    # Reset the index to turn the index into a column
    df.reset_index(inplace=True)

    # Rename the index column to 'Family'
    df.rename(columns={'index': tax}, inplace=True)

    df['Total Train Points'] = df['Total Test Points']/0.15 * 0.75

    tax_df_list[tax] = df

In [ ]:
#Aside: just checking something...
all_species = tax_df_list['species']['species']

# subset to where Timocratica is in all_species
species_names = [s for s in all_species if 'Timocratica' in s]
species_names

In [ ]:
tax_df_list['species']

no_sig = tax_df_list['species']
no_sig = no_sig.loc[(no_sig['Top1 Accuracy'] > 90) & (no_sig['Total Train Points'] > 200)]

print(f'There are {no_sig.shape[0]} species with >90% accuracy and >200 training points')

In [ ]:
# filter the inferences for these species
df_moths = subset_by_species(inference_csvs, no_sig['species'])
df_moths.head()

In [ ]:
crops = pd.DataFrame(df_moths['top_1_species'].value_counts())
crops

In [ ]:
df_moths.head()

# group by top_1_species, subset to 20 random rows with a mix of dep, and image_date
df_moths_subset = df_moths.groupby('top_1_species').apply(lambda x: x.sample(n=min([20, crops['count'][-1]]), random_state=42)).reset_index(drop=True)

In [ ]:
download_dir = f'/gws/nopw/j04/ceh_generic/kgoldmann/{region}_confident_species'

# save the csv
df_moths_subset.to_csv(os.path.join(download_dir, '{region}_confident_species.csv'), index=False)

In [ ]:
output_crops(df_moths_subset, download_dir, region, buffer=5)

# Large Crops

In [34]:
large_moths_backup = subset_by_size(inference_csvs, keep_n=10, moth_only=False, size_cutoff=1000)

reading in the csvs: 100%|██████████| 658/658 [01:49<00:00,  6.01it/s]


In [36]:
large_moths = large_moths_backup.copy()

In [37]:
large_moths = large_moths.sort_values('crop_area', ascending=False)

In [40]:
large_moths = large_moths[large_moths['crop_area'] > 20000]

large_moths.shape

(6500, 45)

In [41]:
large_nonmoths = large_moths.loc[(~ large_moths['order_name'].str.contains('Lepidoptera')) & (large_moths['class_name'] != 'moth'), ]
large_moths = large_moths.loc[large_moths['order_name'].str.contains('Lepidoptera') | (large_moths['class_name'] == 'moth'), ]

In [42]:
download_dir = f'/gws/nopw/j04/ceh_generic/kgoldmann/large_moths/{region}'
os.makedirs(os.path.join(download_dir), exist_ok=True)
output_crops(large_moths.head(100), download_dir, region, buffer=5)

downloading images: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


In [43]:
download_dir = f'/gws/nopw/j04/ceh_generic/kgoldmann/large_nonmoths/{region}'
os.makedirs(os.path.join(download_dir), exist_ok=True)
output_crops(large_nonmoths.head(100), download_dir, region, buffer=5)


downloading images: 100%|██████████| 100/100 [00:52<00:00,  1.91it/s]
